In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

In [3]:
df= pd.read_csv("gaming_total_v2.csv", index_col=0)
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,developpeur_goo,PEGI_goo,developpeur_wiki,PEGI_wiki,Meta_NP,Meta_VP,Meta_NUsers,Meta_VUsers,meta_score,user_review,Rank,test_note_JVC,avis_count_JVC,avis_note_JVC,Classification_Age_JVC,max_joueurs_JVC,Support_JVC,langue_parlée_JVC,texte_JVC,nb_joueurs_JVC,game_mode,game_mode_JVC
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Nintendo,NaN,Nintendo EAD,7.0,76.0,51.0,8.1,539.0,76.0,8.1,1.0,14.0,687.0,13.8,7.0,4.0,DVD,français,français,4.0,2.0,NaN
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,84.0,14.0,8.4,119.0,NaN,NaN,NaN,19.0,3.0,19.0,3.0,NaN,eShop Console Virtuelle Cartouche,NaN,NaN,NaN,NaN,1.0
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,NaN,NaN,Nintendo EAD,3.0,82.0,73.0,8.5,1167.0,82.0,8.4,3.0,16.0,3023.0,17.4,3.0,4.0,DVD,NaN,NaN,4.0,2.0,2.0
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Nintendo,NaN,NaN,NaN,80.0,73.0,8.3,346.0,80.0,8.2,4.0,15.0,581.0,16.5,3.0,4.0,DVD,français,français,4.0,NaN,NaN
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_encode=pd.get_dummies(df)

df_encode.to_csv('df_encode.csv')

In [5]:
df_le=df.copy()
#Encoder les modalités des variables à l'aide d'un LabelEncoder 
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

col_a_sup=df_le.select_dtypes(include='object').columns

for col in df_le.select_dtypes(include='object').columns:
    df_le[f'{col}_encoded'] = le.fit_transform(df[col])

df_le.drop('Unnamed: 0', axis=1)
df_le.drop(col_a_sup, axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
df_le.head()

In [ ]:
for col in df_le.select_dtypes(include=['float', 'int']).columns:
    plt.figure()
    sns.distplot(df_le[col], label=col)
    #sns.distplot(df['Global_Sales'], label= 'Global_Sales')
    plt.legend()

In [ ]:
for col in df_le.select_dtypes(include=['float', 'int']).columns:
    plt.figure()
    sns.histplot(df_le[col], label=col)
    plt.legend()
    